In [1]:
import numpy as np
import zarr
import tifffile
import matplotlib.pyplot as plt
import h5py
from sys import stdout

780-mouse3-1.lsm is an example file and is not included in ViterBrain/data/sample.zip. It is included in this file as an example of how to convert .lsm files to zarr.

## Convert lsm to zarr and save 

In [ ]:
image = tifffile.imread("780-mouse3-1.lsm")
print(image.shape)
image = np.swapaxes(image, 0, 1)
image = image[:, :, 1500:2000, 500:1000]
plt.imshow(image[1, 10, :, :])
print(image.shape)
zarr.save("3-1-soma.zarr", image[1, :, :, :])

## Create h5 image dataset

In [ ]:
with h5py.File("/Users/sejalsrivastava/Desktop/780_Mouse_3/3-1-soma.h5", "w") as f:
    dset = f.create_dataset("image", data=image[:, :, :, :])

### Create masks using Ilastik

## Create probabilities zarr file

In [2]:
with h5py.File(
    "/Users/sejalsrivastava/Desktop/780_Mouse_3/3-1-soma-image_Probabilities.h5", "r"
) as prob:
    probs = prob.get("exported_data")[1, :, :, :]
    h5 = h5py.File(
        "/Users/sejalsrivastava/Desktop/780_Mouse_3/3-1-soma_Probabilities.h5", "w"
    )
    h5.create_dataset("exported_data", data=probs)
    h5.close()
source = h5py.File(
    "/Users/sejalsrivastava/Desktop/780_Mouse_3/3-1-soma_Probabilities.h5", "r"
)
dest = zarr.open(
    "/Users/sejalsrivastava/Desktop/780_Mouse_3/3-1-soma-image_Probabilities.zarr", "w"
)


zarr.copy(source, dest, log=stdout, name="zarr", if_exists="replace")

# contents of zarr file should be accessed using filename.zarr/zarr/exported_data

copy /
copy /exported_data (38, 500, 500) float32
all done: 2 copied, 0 skipped, 38,000,000 bytes copied


(2, 0, 38000000)

## Go to ViterBrain.ipynb to compute labels, states, etc